In [8]:
import os
import json

import pandas as pd
import numpy as np

import torch
from transformers import AutoTokenizer, AutoModel
import faiss

In [ ]:
# 데이터 불러오기
import os, json
import pandas as pd

def load_all_clauses_from_dir(root_dir):
    records = []
    for label in ("유리", "불리"):
        folder = os.path.join(root_dir, label)
        for fname in os.listdir(folder):
            if not fname.endswith(".json"):
                continue
            path = os.path.join(folder, fname)
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
            text = " ".join(data.get("clauseArticle", []))
            basis = " ".join(data.get("illdcssBasiss", [])) if label == "불리" else None
            records.append({
                "filename": fname,
                "label": label,
                "text": text,
                "basis": basis
            })
    return pd.DataFrame(records)

df = load_all_clauses_from_dir("../data/raw")

                  filename label  \
0     001_개인정보취급방침_가공.json    유리   
1      001_결혼정보서비스_가공.json    유리   
2           001_보증_가공.json    유리   
3         001_사이버몰_가공.json    유리   
4         001_상해보험_가공.json    유리   
...                    ...   ...   
7995     620_임대차계약_가공.json    불리   
7996     621_임대차계약_가공.json    불리   
7997     622_임대차계약_가공.json    불리   
7998     623_임대차계약_가공.json    불리   
7999     624_임대차계약_가공.json    불리   

                                                   text  \
0     제2조(개인정보의 처리 및 보유기간) \n① 협회는 법령에 따른 개인정보 보유․이용...   
1     제3조 (회원가입)\n① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사에 제공...   
2     제2조(보증금액)\n ① 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증부...   
3     제3조 (약관 등의 명시와 설명 및 개정)\n① 몰은 이 약관의 내용과 상호 및 대...   
4     제4조(보험금 지급에 관한 세부규정)\n② 제3조(보험금의 지급사유) 제2호에서 장...   
...                                                 ...   
7995  제2조(임대료)\n제2항 임대료 등의 연체시 매월 100분의 10에 해당하는 연체료...   
7996  제12조(해약인정 및 보증금 등의 반환)\n제2항 갑은 해약이 확정되면 즉시 을의 ...   
7997  제17조(임대인의 금지사항)\n제2항 을이 전항 및 제

In [ ]:
# 불러온 데이터 확인
mid_index = len(df) // 2 +2000
print(df.iloc[mid_index:mid_index + 5])


                  filename label  \
6000      026_신용카드_가공.json    불리   
6001       026_예식업_가공.json    불리   
6002     026_온라인게임_가공.json    불리   
6003     026_임대차계약_가공.json    불리   
6004  026_자동차리스및렌트_가공.json    불리   

                                                   text  \
6000  제20조(회원자격정지 및 탈회)\n제1항 카드사는 회원이 다음 각호의 1에 해당되는...   
6001  제10조(계약해지 유보)\n제1항 갑은 을의 권익을 보호하기 위하여 다음의 계약해지...   
6002  제33조 재판권 및 준거법\n제3항 서비스 이용으로 발생한 분쟁에 대해 소송이 제기...   
6003  제 10 조(대금 정산) \n을은 갑에게 계약과 동시 판매기 총금액의 약 30퍼센트...   
6004  제4조(예약의 취소 등)\n제3항 회원이 자신의 사정으로 임차예정 일시 직전 3시간...   

                                                  basis  \
6000  신용카드사가 카드회원의 신용상태의 변화에 따라 회원자격을 정지하거나 카드사용을 일시...   
6001  통상의 거래관행상 위약금은 총거래대금의 10퍼센트인 점과 소비자피해보상규정의 할인회...   
6002       피청구인의 위 약관조항은 고객에 대하여 부당하게 불리한 재판관할의 합의조항이다.   
6003  계약체결 후 기계가 발주됨을 전제로 총 거래대금의 30퍼센트를 손해배상액의 예정으로...   
6004  해당 약관조항은 각 호에 열거된 사유에 해당하는 계약해지 시 대여요금을 반환하지 않...   

                                              embedding  
6000  [0.052592

In [ ]:
# 언어 모델 불러오기
import torch
from transformers import BertModel, AutoTokenizer

MODEL_NAME = "skt/kobert-base-v1"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
model = BertModel.from_pretrained(MODEL_NAME)
model.eval()

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [ ]:
# 텍스트 임베딩 함수
# 자꾸 뻑나서 안 씀
def embed_texts(texts, model, tokenizer, batch_size=16, max_length=512):
    """
    texts: list of str
    returns: np.ndarray of shape (len(texts), hidden_size)
    """
    all_embeds = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        encoded = tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )
        if torch.cuda.is_available():
            encoded = {k:v.to("cuda") for k,v in encoded.items()}
        with torch.no_grad():
            out = model(**encoded)
        cls_embeds = out.last_hidden_state[:, 0, :].cpu().numpy()
        all_embeds.append(cls_embeds)
    return np.vstack(all_embeds)


In [ ]:
# FAISS 인덱스
def build_faiss_index(embeddings: np.ndarray) -> faiss.Index:
    """
    embeddings: (N, D) float32 numpy array
    returns: FAISS index for cosine similarity
    """
    faiss.normalize_L2(embeddings)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    return index


In [ ]:
# 모델이 지원하는 최대 위치 임베딩 길이
# 왜 뻑나는지 확인
max_len = model.config.max_position_embeddings  
max_seq = 0
max_idx = None

for i, text in enumerate(df["text"].fillna("").astype(str)):
    enc = tokenizer(text, return_tensors="pt", truncation=False, padding=False)
    seq_len = enc["input_ids"].size(1)
    if seq_len > max_seq:
        max_seq, max_idx = seq_len, i

print(f"dataset max seq_len = {max_seq} (at idx {max_idx})")
print(f"model.max_position_embeddings = {max_len}")


dataset max seq_len = 993 (at idx 6339)
model.max_position_embeddings = 512


In [ ]:
from sentence_transformers import SentenceTransformer
# 뻑나는 함수 대신 다른 모델 가져와서 임베딩함

# 가벼운 CPU용 모델로 불러오기
sbert = SentenceTransformer("all-MiniLM-L6-v2", device="cpu")

# 텍스트 리스트 준비
texts = df["text"].fillna("").astype(str).tolist()

# 배치 사이즈 32로, 진행바 띄워가며 임베딩 생성
embeddings = sbert.encode(
    texts,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True
)

# float32 변환 후 DataFrame에 추가
df["embedding"] = embeddings.astype("float32").tolist()

print("임베딩 shape:", embeddings.shape)


c:\Users\user\Documents\JSY\KW\3-1\TextMining\test\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not insta

임베딩 shape: (8000, 384)


In [27]:
# 인덱스 구축
index = build_faiss_index(embeddings)
print("FAISS index total vectors:", index.ntotal)

# 첫 번째 항목으로 Top-5 검색
D, I = index.search(embeddings[:1], k=5)
for rank, idx in enumerate(I[0]):
    sim   = D[0][rank]
    label = df.loc[idx, "label"]
    text  = df.loc[idx, "text"][:50] + "…"
    basis = df.loc[idx, "basis"][:50] + "…" if df.loc[idx, "basis"] else ""
    print(f"Rank {rank+1}: (label={label}, sim={sim:.4f})")
    print(f"  text : {text}")
    if basis:
        print(f"  basis: {basis}")
    print()


FAISS index total vectors: 8000
Rank 1: (label=유리, sim=1.0000)
  text : 제2조(개인정보의 처리 및 보유기간) 
① 협회는 법령에 따른 개인정보 보유․이용기간 또는…

Rank 2: (label=유리, sim=0.9307)
  text : 제2조(개인정보의 처리 및 보유기간) 
① 개인정보처리자명은(는) 법령에 따른 개인정보 보…

Rank 3: (label=유리, sim=0.9275)
  text : 제2조(개인정보의 처리 및 보유기간) 
① 개인정보처리자명은 법령에 따른 개인정보 보유, …

Rank 4: (label=유리, sim=0.8989)
  text : 제5조(정보주체와 법정대리인의 권리의무 및 행사방법)
⑤ 개인정보의 정정 및 삭제 요구는 …

Rank 5: (label=유리, sim=0.8846)
  text : 제30조(청구 절차 및 유의 사항) 
④ 보험회사는 손해배상청구에 관한 서류 등을 받았을 …



In [ ]:
# (1) 각 텍스트의 토큰 길이 확인
for i, text in enumerate(texts):
    tokens = tokenizer(text, return_tensors="pt", truncation=False, padding=False)
    print(f"[토큰 길이] idx={i}, tokens.input_ids.shape = {tokens['input_ids'].shape}")

# (2) 임베딩 생성(뻑나는 부분)
embeddings = embed_texts(texts=texts, model=model, tokenizer=tokenizer)

# (3) 반환된 embeddings 리스트의 각 원소 shape 확인
for i, emb in enumerate(embeddings):
    arr = np.array(emb)
    print(f"[임베딩 shape] idx={i}, arr.shape = {arr.shape}")
